In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/data.csv', delimiter=';', decimal=",")

# Преобразование строки в periodStartDate и periodEndDate
df["periodStartDate"] = pd.to_datetime(df["periodStartDate"], format="%d.%m.%Y %H:%M")
df["periodEndDate"] = pd.to_datetime(df["periodEndDate"], format="%d.%m.%Y %H:%M")
# Преобразование periodDuration в секунды
df["periodDuration"] = pd.to_timedelta(df["periodDuration"]).dt.total_seconds()

In [ ]:
#Полносвязная нейронная сеть (DNN/MLP)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Выбор признаков (features) и целевых переменных (targets)
features = df[[
    'minCupolaTemp', 'avgCupolaTemp', 'cupolaTempChangeRate',
    'minSmokeTemp', 'smokeTempChangeRate',
    'minGasFlowRate', 'maxGasFlowRate', 'avgGasFlowRate', 'sumGasFlowRate'
]]
targets = df[['maxCupolaTemp', 'maxSmokeTemp', 'periodDuration']]

# Нормализация данных
scaler_features = StandardScaler()
scaler_targets = StandardScaler()

X = scaler_features.fit_transform(features)
y = scaler_targets.fit_transform(targets)

# Разделение на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Входной слой
    Dense(32, activation='relu'),                                  # Скрытый слой
    Dense(3)                                                       # Выходной слой (3 значения)
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',  # Среднеквадратичная ошибка для регрессии
    metrics=['mae']  # Средняя абсолютная ошибка
)

model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1,
    callbacks=callbacks
)

plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Progress')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Обратная нормализация для интерпретации
y_pred_original = scaler_targets.inverse_transform(y_pred)
y_test_original = scaler_targets.inverse_transform(y_test)

# Пример первых 5 предсказаний
print("Предсказанные значения:\n", y_pred_original[:5])
print("\nРеальные значения:\n", y_test_original[:5])

# Оценка модели
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest MAE DNN/MLP: {test_mae:.2f}")

# Создание графика рассеяния для сравнения предсказанных и реальных значений
plt.figure(figsize=(10, 6))

# График рассеяния
plt.scatter(y_test_original[:, 0], y_pred_original[:, 0], alpha=0.6, label='Предсказанные значения')
plt.plot([y_test_original[:, 0].min(), y_test_original[:, 0].max()],
         [y_test_original[:, 0].min(), y_test_original[:, 0].max()],
         'r--', lw=2)  # Диагональная линия

plt.title('Сравнение предсказанных и реальных значений')
plt.xlabel('Реальные значения')
plt.ylabel('Предсказанные значения')
plt.legend()
plt.grid()
plt.show()